# 원래 type_token_ids는 어떻게 나와야 할까? (확인 완)

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

Reusing dataset glue (/opt/ml/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [2]:
raw_datasets['train']['sentence1'][0]

'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence1", "sentence2", "idx"]
) # 필요 없는 column 제거
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # column 이름 변경
tokenized_datasets.set_format("torch") # format 변경
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [4]:
tokenized_datasets["train"]['token_type_ids'][0]

/opt/conda/lib/python3.8/site-packages/datasets/formatting/formatting.py:147: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [5]:
tokenizer.decode(tokenized_datasets["train"]['input_ids'][0])

'[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP]'

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=False, batch_size=8, collate_fn=data_collator
)

In [7]:
# data processing이 잘 되었는지 하나의 batch로 확인해보기
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 41]),
 'input_ids': torch.Size([8, 41]),
 'labels': torch.Size([8, 41]),
 'token_type_ids': torch.Size([8, 41])}

In [8]:
batch.token_type_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
tokenizer.decode(batch.input_ids[0])

'[CLS] amrozi accused his brother¢ whom he called " [MASK] witness ", of deliberately [MASK]storting his evidence. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

# validation 데이터셋 눈으로 확인하기

In [10]:
import pandas as pd
from datasets import load_from_disk

In [11]:
val_dataset = load_from_disk('/opt/ml/data/train_dataset/validation')

In [12]:
val_dataset

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 240
})

## Validation 데이터셋 - answers

In [14]:
val_dataset['answers']

[{'answer_start': [284], 'text': ['한보철강']},
 {'answer_start': [146], 'text': ['1871년']},
 {'answer_start': [517], 'text': ['나뭇잎']},
 {'answer_start': [1109], 'text': ['금대야']},
 {'answer_start': [386], 'text': ['수평적 관계']},
 {'answer_start': [408], 'text': ['옥음방송']},
 {'answer_start': [192], 'text': ['코칭 스티치']},
 {'answer_start': [31], 'text': ['복잡한 감염병']},
 {'answer_start': [625], 'text': ['스페인']},
 {'answer_start': [71], 'text': ['20세기 초']},
 {'answer_start': [274], 'text': ['"5월의 왕"']},
 {'answer_start': [901], 'text': ["'일급 비밀 프로젝트 2501'"]},
 {'answer_start': [430], 'text': ['테헤란']},
 {'answer_start': [317], 'text': ['역사교육과정개발추진위원회']},
 {'answer_start': [1429], 'text': ['1967년 11월 15일']},
 {'answer_start': [0], 'text': ['1965년']},
 {'answer_start': [162], 'text': ['아리크 부케']},
 {'answer_start': [709], 'text': ['〈중앙일보〉']},
 {'answer_start': [304], 'text': ['미타케성']},
 {'answer_start': [536], 'text': ['전체 4순위']},
 {'answer_start': [689], 'text': ['뇌물']},
 {'answer_start': [78], 'text': [

# Test dataset 확인하기

In [1]:
import pandas as pd
from datasets import load_from_disk

In [2]:
test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation')

In [3]:
test_dataset

Dataset({
    features: ['question', 'id'],
    num_rows: 600
})

In [5]:
test_dataset['question'][7]

'아시노 호가 만들어진 원인은 무엇인가?'

In [6]:
test_dataset['question'][16]

'통대를 뽑을 수 있었던 주체는 누구인가?'